In [1]:
!pip install requests beautifulsoup4
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import csv
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select

# Set up the Chrome driver (use ChromeDriverManager to handle the driver version)
options = Options()
options.headless = False  # Set to True to run in headless mode
service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service, options=options)

# URL of the filtered results page
url = 'https://www.fantasyaftermath.com/#' 

# Open the page with Selenium
driver.get(url)

#initialize an empty list to sore data
player_stats = []

# Wait for the page to fully load (you may need to adjust the wait time or use explicit waits)
time.sleep(5)  

# Open a CSV file to write the data
with open('fantasy_football_data_all_pages.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['Player', 'Position', 'Team', 'Fantasy Diff', 'Actual Fantasy Points', 'Projected Fantasy Points', 
                     'Pass Yards', 'Pass TDs', 'Pass INTs', 'Rush Att', 'Rush Yards', 'Rush TDs', 'Rec Yards', 'Rec TDs', 'Receptions'])

    # Function to scrape the current page's data
    def scrape_page():
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # Find the rows of data (adjust the selector based on the page structure)
        rows = soup.select('div.row.dataRow')  

        # Iterate over each row and extract the relevant data
        for row in rows:
            player = row.select_one('.dpPlayer').get_text(strip=True)
            pos_team = row.select_one('.playerHeader').get_text(strip=True)
            fantasy_diff = row.select_one('.dpDiff').get_text(strip=True)
            actual_points = row.select_one('.fantasyStandard .dataLeft').get_text(strip=True)
            projected_points = row.select_one('.fantasyStandard .dataRight').get_text(strip=True)

            # Extract stats like passing yards, rushing yards, receptions, etc.
            pass_yds = row.select_one('.dpPassingYds .spanData').get_text(strip=True)
            pass_tds = row.select_one('.dpPassingTds .spanData').get_text(strip=True)
            pass_ints = row.select_one('.dpPassingInts .spanData').get_text(strip=True)
            rush_att = row.select_one('.dpRushingAtt .spanData').get_text(strip=True)
            rush_yds = row.select_one('.dpRushingYds .spanData').get_text(strip=True)
            rush_tds = row.select_one('.dpRushingTds .spanData').get_text(strip=True)
            rec_yds = row.select_one('.dpReceivingYds .spanData').get_text(strip=True)
            rec_tds = row.select_one('.dpReceivingTds .spanData').get_text(strip=True)
            recs = row.select_one('.dpReceivingRec .spanData').get_text(strip=True)

            # Write the data row to the CSV file
            writer.writerow([player, pos_team, fantasy_diff, actual_points, projected_points, pass_yds, pass_tds, pass_ints, 
                             rush_att, rush_yds, rush_tds, rec_yds, rec_tds, recs])

    # Scrape the first page
    scrape_page()

    # Interact with the dropdown filter to change weeks or seasons
    week_select = Select(driver.find_element(By.ID, 'weekSelect'))

    # Loop through each option in the dropdown and scrape data for each selection
    for option in week_select.options:
        # Skip the already selected option (optional)
        if option.get_attribute('selected') == 'true':
            continue

        # Select the current option
        week_select.select_by_value(option.get_attribute('value'))
        print(f"Scraping data for {option.text}...")

        # Wait for the page to reload with the new filter
        time.sleep(3)  
        # Scrape the page's data for the selected week/season
        scrape_page()

        # Check for pagination (Next page button)
        while True:
            try:
                # Find the "Next" button (adjust the selector for your page's "Next" button)
                next_button = driver.find_element(By.CSS_SELECTOR, '.pagination-next a')  
                if not next_button.is_enabled():  
                    break

                # Click the "Next" button to go to the next page
                next_button.click()
                time.sleep(3) 
               
                # Scrape the new page
                scrape_page()

            except Exception as e:
                print("Error occurred while navigating to next page:", e)
                break  
if player_stats:
    player_df = pd.DataFrame(player_stats, columns = ['player', 'pos_team', 'fantasy_diff', 'actual_points', 'projected_points', 'pass_yds', 'pass_tds', 'pass_ints', 
                             'rush_att', 'rush_yds', 'rush_tds', 'rec_yds', 'rec_tds', 'recs'])
    print(player_df.head())
    player_df.to_csv('fantasy_football_data_all_pages.csv', index = False)
    
    
# Close the WebDriver after scraping all pages
driver.quit()

print("Data has been written to fantasy_football_data_all_pages.csv")


C:\Users\nschn\AppData\Local\Temp\ipykernel_15500\542496170.py:18: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False  # Set to True to run in headless mode


Scraping data for Week 1...
Error occurred while navigating to next page: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pagination-next a"}
  (Session info: chrome=131.0.6778.140)
Stacktrace:
	GetHandleVerifier [0x00C434A3+25059]
	(No symbol) [0x00BCCEA4]
	(No symbol) [0x00AABEC3]
	(No symbol) [0x00AEFD86]
	(No symbol) [0x00AEFFCB]
	(No symbol) [0x00B2D952]
	(No symbol) [0x00B11F44]
	(No symbol) [0x00B2B51E]
	(No symbol) [0x00B11C96]
	(No symbol) [0x00AE3FAC]
	(No symbol) [0x00AE4F3D]
	GetHandleVerifier [0x00F35613+3113811]
	GetHandleVerifier [0x00F4A2DA+3199002]
	GetHandleVerifier [0x00F42AB2+3168242]
	GetHandleVerifier [0x00CE3310+680016]
	(No symbol) [0x00BD57ED]
	(No symbol) [0x00BD2A98]
	(No symbol) [0x00BD2C35]
	(No symbol) [0x00BC5890]
	BaseThreadInitThunk [0x76FD7BA9+25]
	RtlInitializeExceptionChain [0x779BC0CB+107]
	RtlClearBits [0x779BC04F+191]

Scraping data for Week 2...
Error occurred while navigating to next page: Message: no su